In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import os
from urllib.request import urlopen, quote,URLError, HTTPError
import time

In [3]:
inputpath_urls = "G:\\work\\logistica\\stations\\JDStations\\yuanURL\\浙江/浙江.xlsx"
inputpath_citys = "G:\\work\\logistica\\stations\\JDStations\\stations\\zhejiangcityID.xlsx"

# 导入数据
df = pd.read_excel(inputpath_urls)
data_city = pd.read_excel(inputpath_citys)
province = "浙江省"

In [4]:
urls = []
for a in df["url"]:
    urls.append("http://www.jdwl.com/site/querySiteList?" + a)

print("验证完整地址：")
print(urls[0:3])

验证完整地址：
['http://www.jdwl.com/site/querySiteList?provinceId=15&cityId=1158&countyId=1224', 'http://www.jdwl.com/site/querySiteList?provinceId=15&cityId=1158&countyId=1226', 'http://www.jdwl.com/site/querySiteList?provinceId=15&cityId=1158&countyId=1227']


In [5]:
def get_JD_siteInfo(urls):
    JD_siteList = []
    for url in urls:
        request = requests.get(url)
        soup = BeautifulSoup(request.content, "lxml").get_text()
        jsonlist = json.loads(soup)
        for i in range(len(jsonlist)):
            JD_siteDict = {}
            JD_siteDict["JD_siteBusinessName"] = jsonlist[i]["siteBusinessName"]
            JD_siteDict["JD_siteProvinceId"] = jsonlist[i]["provinceId"]
            JD_siteDict["JD_siteCityId"] = jsonlist[i]["cityId"]
            JD_siteDict["JD_siteCountyId"] = jsonlist[i]["countyId"]
            JD_siteDict["JD_siteName"] = jsonlist[i]["siteName"]
            if 'address' in jsonlist[i]:
                JD_siteDict["JD_siteAddress"] = jsonlist[i]["address"]
            else:
                pass
            if 'latitude' in jsonlist[i]:
                JD_siteDict["JD_siteLatitude"] = jsonlist[i]['latitude']
            else:
                pass  
            if 'longitude' in jsonlist[i]:
                JD_siteDict["JD_siteLongitude"] = jsonlist[i]['longitude']
            else:
                pass       
            if "telephone" in jsonlist[i]:
                JD_siteDict["JD_siteTelephone"] = jsonlist[i]["telephone"]
            else:
                pass
            JD_siteList.append(JD_siteDict)
    return JD_siteList

In [8]:
df_JD_siteInfo = pd.DataFrame(get_JD_siteInfo(urls))
# 去除空白地址所在行
print("去除空白地址前：")
print(df_JD_siteInfo['JD_siteAddress'].isnull().value_counts())
df_JD_siteInfo = df_JD_siteInfo.dropna(subset=["JD_siteAddress"])
print("去除空白地址后：")
print(df_JD_siteInfo['JD_siteAddress'].isnull().value_counts())
print("*"*50)
print("查看JD网店及对应城市列名：")
print(df_JD_siteInfo.columns)
print("$"*30)
print(data_city.columns)
print("*"*50)

去除空白地址前：
False    447
True       1
Name: JD_siteAddress, dtype: int64
去除空白地址后：
False    447
Name: JD_siteAddress, dtype: int64
**************************************************
查看JD网店及对应城市列名：
Index(['JD_siteAddress', 'JD_siteBusinessName', 'JD_siteCityId',
       'JD_siteCountyId', 'JD_siteLatitude', 'JD_siteLongitude', 'JD_siteName',
       'JD_siteProvinceId', 'JD_siteTelephone'],
      dtype='object')
$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
Index(['cityId', 'cityName'], dtype='object')
**************************************************


In [13]:
data_city['cityId'] = data_city['cityId'].map(lambda x:int(x))

In [14]:
df_JD_siteInfo["JD_siteCityId"] = df_JD_siteInfo["JD_siteCityId"].map(lambda x:int(x))

In [23]:
df_JD_siteInfo = pd.merge(df_JD_siteInfo,data_city,how="left",left_on="JD_siteCityId",right_on="cityId")

In [21]:
df_JD_siteInfoA = pd.concat(df_JD_siteInfo,a)

TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"

In [22]:
a.to_excel(r"G:\work\logistica\stations\JDStations\stations\text1.xlsx")

In [24]:
df_JD_siteInfo.to_excel(r"G:\work\logistica\stations\JDStations\stations\text.xlsx")

In [ ]:
print(df_JD_siteInfo[["JD_siteCityId","JD_cityName"]])

In [ ]:
print(data.columns)
print("*"*30)
print(data_city.columns)

In [ ]:
# df = data[['订单编号','网点地址']]
# df = data.copy()

In [ ]:
df = pd.merge(data,data_city,how="left",on="cityId")

In [ ]:
df.columns

In [ ]:
def getlnglat(address):
    url = 'http://api.map.baidu.com/geocoder/v2/'
    output = 'json'#输出数据的格式	
    ak = 'z3KEtliyTGvj0bFudEkz4GO0GN8eQQa5'
    add = quote(address) #由于本文地址变量为中文，为防止乱码，先用quote进行编码
    uri = url + '?' + 'address=' + add  + '&output=' + output + '&ak=' + ak 
    req = urlopen(uri)
    res = req.read().decode() 
    temp = json.loads(res)
    return temp

In [ ]:
print("地址去空前：{}".format(df['siteAddress'].isnull().value_counts()))
      
df = df.dropna(subset=["siteAddress"])
      


In [ ]:
print("地址去空后：{}".format(df['siteAddress'].isnull().value_counts()))

In [ ]:
df["siteAddress_all"] = province+ df['cityName'] + df['siteAddress']

In [ ]:
df["siteAddress_all"]

In [ ]:
start_time = time.time()
listA = []
for b in df['siteAddress_all']:
#     print(b)
    dictA = {}
    try:
        temp = getlnglat(b)
    except HTTPError as e:
        print("请求出错")
        pass
    else:    
        if ('result' in temp):
            dictA["siteLongitude_baidu"] = temp['result']['location']['lng']
            dictA["siteLatitude_baidu"] = temp['result']['location']['lat']
            dictA["sitePrecise_baidu"] = temp['result']['precise']
            dictA["siteConfidence_baidu"] = temp['result']['confidence']
            dictA["siteComprehension_baidu"] = temp['result']['comprehension']
            dictA["siteLevel_baidu"] = temp['result']['level']
        else:
            pass
    listA.append(dictA)
#     time.sleep(1)
end_time = time.time()
print("总用时：",end_time-start_time)

0.857048749923706
5.161295413970947


In [ ]:
listA

In [ ]:
df_A = pd.DataFrame(listA)

In [ ]:
df_all = pd.concat([df,df_A],axis=1,join_axes=[df.index])

In [ ]:

print("data:{}行".format(data.shape[0]))
print("df:{}行".format(df.shape[0]))
print("df_A:{}行".format(df_A.shape[0]))
print("df_all:{}行".format(df_all.shape[0]))

In [ ]:
df_A.columns

In [ ]:
df_all.columns

In [ ]:
df_all.to_excel(outpath + name_ANSI)
# df_all.to_csv(outpath + name_UTF8, index=False)